<a href="https://colab.research.google.com/github/buabaj/Mobile-data-outage-prediction/blob/main/CD3_corrected.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [20]:
data = pd.read_excel('SD3.xlsx')

In [21]:
data.isnull().sum()

REGION       0
 CAUSE       0
 2G          0
3G           0
4G           0
CATEGORY     0
REFERENCE    0
DATE         0
MTTR         0
MONTH        0
dtype: int64

In [22]:
!pip install xplore
from xplore.data import xplore

In [23]:
xplore(data)

------------------------------------
The fist 5 entries of your dataset are:

   REGION   CAUSE   2G   3G   ...         REFERENCE      DATE         MTTR   MONTH
0       8       6     4    1  ...  2018010100000026  20180101    75.000000  201801
1       8       6     4    1  ...  2018010100000051  20180101   292.000000  201801
2       4       6     2    0  ...  2018010100000056  20180101  2307.000000  201801
3       4       8     1    1  ...  2018010100000076  20180101   265.733333  201801
4       5       4     5    2  ...  2018010100000079  20180101    33.000000  201801

[5 rows x 10 columns]


------------------------------------
The last 5 entries of your dataset are:

      REGION   CAUSE   2G   3G   ...         REFERENCE      DATE       MTTR   MONTH
1993       9       6    18   12  ...  2018081800000135  20180818   4.800000  201808
1994       1       6    13    0  ...  2018081800000158  20180818  94.083333  201808
1995       7       6    36   27  ...  2018081900000032  20180819   6.

In [24]:
df = pd.DataFrame(data)
df

,REGION,CAUSE,2G,3G,4G,CATEGORY,REFERENCE,DATE,MTTR,MONTH
0,8,6,4,1,0,1,2018010100000026,20180101,75.000000,201801
1,8,6,4,1,0,1,2018010100000051,20180101,292.000000,201801
2,4,6,2,0,0,1,2018010100000056,20180101,2307.000000,201801
3,4,8,1,1,0,1,2018010100000076,20180101,265.733333,201801
4,5,4,5,2,0,0,2018010100000079,20180101,33.000000,201801
...,...,...,...,...,...,...,...,...,...,...
1993,9,6,18,12,4,0,2018081800000135,20180818,4.800000,201808
1994,1,6,13,0,0,0,2018081800000158,20180818,94.083333,201808
1995,7,6,36,27,9,0,2018081900000032,20180819,6.066667,201808
1996,9,8,7,6,0,0,2018081900000040,20180819,5.950000,201808


In [25]:
X = df.drop('REFERENCE', axis='columns')

In [26]:
y = df[['MTTR']].astype(float).astype(int)

In [27]:
y

,MTTR
0,75
1,292
2,2307
3,265
4,33
...,...
1993,4
1994,94
1995,6
1996,5


In [28]:
seed = 50  # so that the result is reproducible
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state = seed)

In [29]:
X_train.dtypes

REGION        int64
 CAUSE        int64
 2G           int64
3G            int64
4G            int64
CATEGORY      int64
DATE          int64
MTTR        float64
MONTH         int64
dtype: object

In [30]:
features_to_encode = list(X_train.select_dtypes(include = ['object']).columns)

In [31]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
col_trans = make_column_transformer(
                        (OneHotEncoder(),features_to_encode),
                        remainder = "passthrough"
                        )

In [32]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier(
                      min_samples_leaf=50,
                      n_estimators=150,
                      bootstrap=True,
                      oob_score=True,
                      n_jobs=-1,
                      random_state=seed,
                      max_features='auto')

In [33]:
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(col_trans, rf_classifier)
np.array(X_train, dtype=np.float32)
pipe.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)


Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories='auto',
                                                                drop=None,
                                                                dtype=<class 'numpy.float64'>,
                                                                handle_unknown='error',
                                                                sparse=True),
                                                  [])],
                                   verbose=False)),
                ('randomforestclassifier',
                 RandomForestClas...ap=True, ccp_alpha=0.0,
                                        class_weight